<a href="https://colab.research.google.com/github/butter-my-toast/RockPaperScissors-CNN/blob/main/rps_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.nn.functional as F
import os
import cv2
from google.colab import files
import matplotlib.pyplot as plt
from random import shuffle
import matplotlib.image as mpimg
import numpy as np
import shutil
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, ConcatDataset

In [3]:
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d sanikamal/rock-paper-scissors-dataset
!unzip -q rock-paper-scissors-dataset.zip

Saving kaggle.json to kaggle.json
 99% 447M/452M [00:06<00:00, 152MB/s]
100% 452M/452M [00:06<00:00, 74.3MB/s]


In [113]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.437, 0.503, 0.51], std=[0.243, 0.254, 0.232]),
    transforms.RandomGrayscale(0.2),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224), antialias=True)
])



In [114]:
train_set = ImageFolder('/content/Rock-Paper-Scissors/train', transform=transform)
test_set = ImageFolder('/content/Rock-Paper-Scissors/test', transform=transform)

In [127]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=True)
for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 3, 224, 224])
Shape of y: torch.Size([64]) torch.int64


In [128]:
len(train_dataloader)

40

--------------




In [129]:

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 3)
        self.bn1 = nn.BatchNorm2d(6) 
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 4)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*54*54, 380)
        self.dropout1 = nn.Dropout(0.4)
        self.fc2 = nn.Linear(380, 160)
        self.dropout2 = nn.Dropout(0.4)
        self.fc3 = nn.Linear(160, 3)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x)))) 
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = x.view(-1, 16*54*54)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


cnn = CNN().to(device)
print(cnn)

Using cuda device
CNN(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(4, 4), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=46656, out_features=380, bias=True)
  (dropout1): Dropout(p=0.4, inplace=False)
  (fc2): Linear(in_features=380, out_features=160, bias=True)
  (dropout2): Dropout(p=0.4, inplace=False)
  (fc3): Linear(in_features=160, out_features=3, bias=True)
)


--------------




In [130]:
loss_fn = nn.CrossEntropyLoss()
optimizer_cnn = torch.optim.Adam(cnn.parameters(), lr=1e-4)

In [131]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [132]:
def test(dataloader, model, loss_fn, mode='test'):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # print("num correct: " + str(correct) + " with label " + str(y))
    test_loss /= num_batches
    correct /= size
    print(f"{mode} Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [133]:
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, cnn, loss_fn, optimizer_cnn)
    test(train_dataloader, cnn, loss_fn, mode='train')
    test(test_dataloader, cnn, loss_fn)

Epoch 1
-------------------------------
loss: 1.094363  [   64/ 2520]
loss: 0.841246  [  704/ 2520]
loss: 0.609638  [ 1344/ 2520]
loss: 0.400149  [ 1984/ 2520]
train Error: 
 Accuracy: 96.6%, Avg loss: 0.273727 

test Error: 
 Accuracy: 67.5%, Avg loss: 0.722139 

Epoch 2
-------------------------------
loss: 0.266808  [   64/ 2520]
loss: 0.155264  [  704/ 2520]
loss: 0.236434  [ 1344/ 2520]
loss: 0.115161  [ 1984/ 2520]
train Error: 
 Accuracy: 99.3%, Avg loss: 0.047848 

test Error: 
 Accuracy: 81.2%, Avg loss: 0.477350 

Epoch 3
-------------------------------
loss: 0.089445  [   64/ 2520]
loss: 0.102936  [  704/ 2520]
loss: 0.025292  [ 1344/ 2520]
loss: 0.078544  [ 1984/ 2520]
train Error: 
 Accuracy: 99.9%, Avg loss: 0.015893 

test Error: 
 Accuracy: 74.7%, Avg loss: 0.546995 

Epoch 4
-------------------------------
loss: 0.078910  [   64/ 2520]
loss: 0.090360  [  704/ 2520]
loss: 0.021225  [ 1344/ 2520]
loss: 0.029443  [ 1984/ 2520]
train Error: 
 Accuracy: 100.0%, Avg loss: 0.

--------------




In [134]:
torch.save(cnn.state_dict(), "model.pth")
print("pytorch model saved to model.pth")

pytorch model saved to model.pth


In [ ]:
model = cnn().to(device)
model.load_state_dict(torch.load("model.pth"))